In [9]:
# Load all the audio file paths in a Google Drive folder
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

Mounted at /content/drive


In [10]:
!pip install transformers
!pip install accelerate

In [11]:
import os
import librosa
import soundfile as sf
from google.colab import drive
import json

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:

# This will create the WhisperVideo files if they don't exist.
folders =  ["WhisperVideo/Video", "WhisperVideo/TextFiles/", "WhisperVideo/AudioFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)


In [16]:
for video_file_name in os.listdir("/content/drive/MyDrive/WhisperVideo/Video"):

  video_path = r"/content/drive/MyDrive/WhisperVideo/Video/" + video_file_name
  audio_path = r"/content/drive/MyDrive/WhisperVideo/AudioFiles/" + video_file_name[:-4] + ".wav" # Replace the video extension with .wav
  save_audio_descrption = r"/content/drive/MyDrive/WhisperVideo/TextFiles/" + video_file_name[:-4] + ".txt"

  y, sr = librosa.load(video_path, sr=16000) # Load the audio with 16 kHz sampling rate
  sf.write(audio_path, y, sr) # Save the audio as a wav file

  result = pipe(audio_path,
                return_timestamps=True,
                generate_kwargs={"language": "italian", "task": "translate"})

  with open(save_audio_descrption, "w") as f:
    json.dump( result, f)

<ipython-input-16-0591bdf15765>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(video_path, sr=16000) # Load the audio with 16 kHz sampling rate
<ipython-input-16-0591bdf15765>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(video_path, sr=16000) # Load the audio with 16 kHz sampling rate
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
